# Notebook de la dimensión Ciudad - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [3]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2. Extracción de datos desde la base de datos - Extract

In [4]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [5]:
# Lectura de tablas ciudad y departamento
df_ciudad = pd.read_sql("SELECT * FROM ciudad;", engine_fuente)
df_departamento = pd.read_sql("SELECT * FROM departamento;", engine_fuente)

## 3. Transformación de los datos - Transform

In [6]:
# Renombrar columnas
df_departamento.rename(columns={"nombre": "nombre_departamento"}, inplace=True)
df_ciudad.rename(columns={"nombre": "nombre_ciudad"}, inplace=True)

In [7]:
# Unir ciudad con departamento
dim_ciudad = df_ciudad.merge(df_departamento, on="departamento_id", how="left")

In [9]:
from datetime import datetime

# Agregar columna de fecha de carga
dim_ciudad["fecha_carga"] = datetime.today().date()

In [10]:
# Seleccionar y renombrar columnas
dim_ciudad = dim_ciudad[["ciudad_id", "nombre_ciudad", "nombre_departamento", "fecha_carga"]]
dim_ciudad.rename(columns={"ciudad_id": "id_ciudad"}, inplace=True)

In [11]:
# Verificar resultado
dim_ciudad.head()

,id_ciudad,nombre_ciudad,nombre_departamento,fecha_carga
0,6,BUGA,VALLE DEL CAUCA,2025-06-05
1,5,BOGOTA,CUNDINAMARCA,2025-06-05
2,4,PASTO,NARIÑO,2025-06-05
3,3,POPAYAN,CAUCA,2025-06-05
4,2,PALMIRA,VALLE DEL CAUCA,2025-06-05


## 4. Guardado de la dimensión - Load

In [12]:
# Guardar como CSV
dim_ciudad.to_csv("dim_ciudad.csv", index=False)

In [13]:
# Cargar en la bodega de datos OLAP
dim_ciudad.to_sql("dim_ciudad", engine_bodega, if_exists="replace", index=False)

19

In [14]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_ciudad.csv")
print(df_verificacion.head())

   id_ciudad nombre_ciudad nombre_departamento fecha_carga
0          6          BUGA     VALLE DEL CAUCA  2025-06-05
1          5        BOGOTA        CUNDINAMARCA  2025-06-05
2          4         PASTO              NARIÑO  2025-06-05
3          3       POPAYAN               CAUCA  2025-06-05
4          2       PALMIRA     VALLE DEL CAUCA  2025-06-05
